# Development of Dashboard
## SPACE-X Data

____

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

    Which site has the largest successful launches?
    Which site has the highest launch success rate?
    Which payload range(s) has the highest launch success rate?
    Which payload range(s) has the lowest launch success rate?
    Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?


### Import required libraries

In [3]:
import requests

import pandas as pd
import numpy as np

import plotly.express as px

from jupyter_dash import JupyterDash

from dash import html
from dash import dcc
from dash.dependencies import Input, Output

# Completion message
print('Required libaries imported\n')

Required libaries imported



Load template code and transfer to Jupyter Notebook

import urllib.request
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
filename = 'spacex_dash_app.py'
urllib.request.urlretrieve(url, filename)

### Load an clean data

In [4]:
spaceX_launch_dash_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(spaceX_launch_dash_url)
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [5]:
def success_label(c):
    if c==1:
        return 'Success'
    else:
        return 'Failure'
        
spacex_df['Launch Outcome'] = spacex_df.apply(lambda x: success_label(x['class']),axis=1)
spacex_df.tail()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category,Launch Outcome
51,51,48,CCAFS SLC-40,0,4230.0,F9 FT B1032.2,FT,Failure
52,52,50,CCAFS SLC-40,0,6092.0,F9 B4 B1044,B4,Failure
53,53,52,CCAFS SLC-40,0,2647.0,F9 B4 B1039.2,B4,Failure
54,54,53,CCAFS SLC-40,1,362.0,F9 B4 B1045.1,B4,Success
55,55,56,CCAFS SLC-40,0,5384.0,F9 B4 B1040.2,B4,Failure


Calculate useful variables

In [8]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

### Create DASH app

In [14]:
# Create a dash application
app = JupyterDash(__name__)


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(id='site-dropdown',options=
                                             [{'label':x , 'value':x} for x in spacex_df['Launch Site'].unique()] 
                                            +[{'label':'All Sites' , 'value':'ALL'}],
                                             value = 'ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,
                                                         marks={1000*i: '{}'.format(1000*i) for i in range(11)},
                                                         value=[min_payload, max_payload]
                                                        )
                                        ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    cdm = {0:'rgb(255,48,48)',1:'rgb(0,201,87)'}
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
                     names='Launch Site', 
                     title='Total Success Launches by Site')
        return fig
    else:
        filtered_df = filtered_df[filtered_df['Launch Site']==entered_site]
        data = (filtered_df.groupby(["class"])
                .size()
                .reset_index(name="Counts")
               )                
        fig = px.pie(data, values='Counts', 
                     names='class', 
                     title='Total Success Launches for site '+ entered_site,
                     color='class',
                     color_discrete_map=cdm)
        fig.update_traces(textposition='inside', 
                     text=data['Counts'].map("{:,} launches".format),
                     textinfo='percent+text')
        fig.update_traces(sort=False) 
        return fig

        
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")],
             )
def get_payload_chart(entered_site,payload):  #payload is list with min and max 'ALL'
    filtered_df = spacex_df[((spacex_df['Payload Mass (kg)']>payload[0]) & (spacex_df['Payload Mass (kg)']<payload[1]))]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', 
                     title='Correlation between Payload and Success for '+entered_site)
        return fig
    else:
        filtered_df=filtered_df[filtered_df['Launch Site']==entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', 
                     title='Correlation between Payload and Success for '+entered_site)
        return fig

        
        
# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, dev_tools_ui=False, dev_tools_props_check=False)

C:\Users\Fowle\Anaconda\envs\my_env\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

127.0.0.1 - - [07/Sep/2022 16:36:40] "GET /_shutdown_77a7934f-034f-4bee-8149-e2710d8b1039 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2022 16:36:40] "GET /_alive_77a7934f-034f-4bee-8149-e2710d8b1039 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [07/Sep/2022 16:36:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:36:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:36:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:36:48] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Sep/2022 16:36:48] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Sep/2022 16:36:48] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Sep/2022 16:36:48] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Sep/2022 16:36:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:36:48] "POST /_dash-update-component HTTP/1.1" 200 -


After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

    Which site has the largest successful launches?
        -Kennedy Space Center
    Which site has the highest launch success rate?
        -Kennedy Space Center
    Which payload range(s) has the highest launch success rate?
        The highest success rate was seen for a payload range of 7000-9000 kg with 1:1
        The highest significant success rate was seen for a payload range of 3000-4000 kg with 7:3
        
    Which payload range(s) has the lowest launch success rate?
    Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate
        -FT has the highest launch success rate

In [15]:
spacex_df.groupby(["Launch Outcome"]).size().reset_index(name="Counts")

spacex_df[spacex_df['Launch Site']=='CCAFS LC-40'].groupby(["Launch Outcome"]).size().reset_index(name="Counts")

,Launch Outcome,Counts
0,Failure,19
1,Success,7


127.0.0.1 - - [07/Sep/2022 17:03:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:03:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:03:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:03:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:04:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:04:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:04:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:04:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:04:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:05:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:05:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 17:05:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.